# Continuous Answer Type

This notebook demonstrates how to generate forecasting questions with numeric/continuous answers. Continuous questions are perfect for quantitative predictions like sports statistics, financial metrics, or any numeric outcome.

In [ ]:
%pip install lightningrod-ai

from IPython.display import clear_output
clear_output()

In [ ]:
import os
from datetime import datetime, timedelta
from lightningrod import (
    LightningRod,
    NewsSeedGenerator,
    QuestionGenerator,
    QuestionPipeline,
    WebSearchLabeler,
    FilterCriteria,
    AnswerType,
    AnswerTypeEnum,
)
from lightningrod._generated.models import QuestionRenderer

api_key = os.getenv("LIGHTNINGROD_API_KEY", "your-api-key-here")

client = LightningRod(api_key=api_key)

## Continuous Answer Type Overview

Continuous questions return numeric values. They're ideal for:
- Sports statistics (points, rebounds, assists)
- Financial predictions (stock prices, revenue)
- Metrics forecasting (user growth, sales numbers)
- Any quantitative outcome

Continuous questions work with any data source: Google News, GDELT, or custom documents.

In [ ]:
answer_type = AnswerType(answer_type=AnswerTypeEnum.CONTINUOUS)

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    interval_duration_days=7,
    search_query=[
        "NBA player statistics",
        "stock market prices",
        "company earnings reports",
    ],
    articles_per_search=15,
)

In [ ]:
question_generator = QuestionGenerator(
    instructions=(
        "Generate forecasting questions that ask for specific numeric values. "
        "Questions should be answerable with a single number."
    ),
    examples=[
        "How many points will LeBron James score in his next game?",
        "What will Apple's stock price be at market close on Friday?",
        "How many users will the app have by the end of Q2?",
        "What will Tesla's revenue be for Q1 2025?",
    ],
    bad_examples=[
        "Will LeBron score over 25 points?",
        "Is Apple's stock price high?",
        "Will the app be successful?",
    ],
    filter_=FilterCriteria(
        rubric="The question must ask for a specific numeric value",
        min_score=0.7
    ),
    answer_type=answer_type,
)

In [ ]:
labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

renderer = QuestionRenderer(answer_type=answer_type)

## Run the Pipeline

Generate continuous questions from the news articles.

In [ ]:
pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=renderer,
)

dataset = client.transforms.run(pipeline_config, max_questions=30)

## Interpreting Continuous Labels

Continuous labels are numeric values with confidence scores. The confidence indicates how certain the labeler is about the accuracy of the number.

In [ ]:
samples = dataset.to_samples()

print(f"Generated {len(samples)} continuous questions\n")

for i, sample in enumerate(samples[:5]):
    print(f"Question {i+1}: {sample.question.question_text if sample.question else 'N/A'}")
    if sample.label:
        label_value = sample.label.label
        try:
            numeric_value = float(label_value)
            print(f"  Answer: {numeric_value}")
        except (ValueError, TypeError):
            print(f"  Answer: {label_value}")
        print(f"  Confidence: {sample.label.label_confidence:.2f}")
        if sample.label.label_confidence > 0.8:
            print("  ✓ High confidence")
        elif sample.label.label_confidence > 0.5:
            print("  ~ Medium confidence")
        else:
            print("  ? Low confidence")
    print()

## Use Cases for Continuous Questions

**Sports Statistics:**
- "How many points will Player X score?"
- "How many rebounds will Team Y grab?"

**Financial Predictions:**
- "What will Stock X's price be?"
- "What will Company Y's revenue be?"

**Metrics Forecasting:**
- "How many users will the app have?"
- "What will the conversion rate be?"